# Graph Components and Implementation with LangGraph

This notebook demonstrates how to build **stateful execution graphs**
using **LangGraph**.

LangGraph allows complex workflows to be expressed as graphs where:
- Nodes operate on a shared state
- Edges control execution flow
- Conditional routing enables dynamic behavior

The focus here is on:
- Defining state
- Creating nodes
- Connecting nodes with edges
- Adding conditional execution paths


In [3]:
import getpass
import os
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage, BaseMessage
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.runnables import Runnable
from collections.abc import Sequence
from typing import Literal

In [4]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Defining the Graph State

LangGraph nodes communicate by reading from and writing to
a shared **state** object.

The state is defined using a `TypedDict` for clarity and type safety.


In [5]:
class State(TypedDict):
    messages: Sequence[BaseMessage]

## Initial State

An initial state is created with a single user message.
This state will be passed through the graph.


In [6]:
state = State(messages = [HumanMessage("Could you give me a grook by Piet Hein ~ 90 words?")])

In [7]:
state["messages"][0].pretty_print()

## Chat Model Initialization

A deterministic chat model is used as the core reasoning component
within graph nodes.


In [8]:
chat = ChatOpenAI(
    model="gpt-5-nano", 
    temperature=0, 
    model_kwargs= {"text":{"verbosity": 'low'}},
    ) 


## Defining Graph Nodes

Each node is a function that:
- Accepts the current state
- Performs some computation
- Returns an updated state


In [9]:
response = chat.invoke(state["messages"])
print(response.text)

In [10]:
def chatbot(state: State) -> State:
    
    print(f"\n-------> ENTERING chatbot:")
    
    response = chat.invoke(state["messages"])
    response.pretty_print()
    
    return State(messages = [response])

## Testing a Node Independently

Nodes can be tested outside the graph
to verify their behavior.



In [11]:
chatbot(state)

In [12]:
graph = StateGraph(State)

In [13]:
graph.add_node("chabot", chatbot)
graph.add_edge(START, "chabot")
graph.add_edge("chabot", END)

In [14]:
graph_compiled = graph.compile()

## Compiled Graph as a Runnable

Compiled graphs implement the `Runnable` interface
and can be invoked like other LangChain components.


In [15]:
isinstance(graph_compiled, Runnable)

In [16]:
graph_compiled

In [17]:
graph_compiled.invoke(state)

## Conditional Graph Execution

LangGraph supports conditional routing,
allowing the execution path to change dynamically
based on the current state.


### Nodes with User Interaction

The following nodes request user input
to demonstrate dynamic graph behavior.


In [18]:
def ask_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_question:")
    
    print("What is your question?")
    
    return State(messages = [HumanMessage(input())])

In [19]:
ask_question(State(messages = []))

In [20]:
def ask_another_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_another_question:")
    
    print("Would you like to ask one more question (yes/no)?")
    
    return State(messages = [HumanMessage(input())])

In [21]:
ask_another_question(State(messages = []))

## Defining a Routing Function

Routing functions determine the next node
based on the current state.


In [22]:
def routing_function(state: State) -> Literal["ask_question", "__end__"]:
    
    if state["messages"][0].content == "yes":
        return "ask_question"
    else:
        return "__end__"

## Graph with Conditional Edges

The graph is extended with multiple nodes
and conditional execution paths.


In [23]:
graph = StateGraph(State)

In [24]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("ask_another_question", ask_another_question)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "ask_another_question")
graph.add_conditional_edges(source = "ask_another_question", 
                            path = routing_function)

In [25]:
graph_compiled = graph.compile()

In [26]:
graph_compiled

## Visualizing the Graph

LangGraph provides an ASCII visualization
to inspect execution flow.


In [27]:
print(graph_compiled.get_graph().draw_ascii())

## Executing the Graph

The compiled graph is invoked with an initial state.
Execution follows the defined edges and routing logic.


In [28]:
graph_compiled.invoke(State(messages = []))

## Summary

This notebook demonstrated:

- Defining shared state with `TypedDict`
- Creating graph nodes that operate on state
- Building execution graphs with `StateGraph`
- Adding conditional routing for dynamic execution
- Visualizing and executing LangGraph workflows

LangGraph enables expressive, stateful,
and controllable LLM-driven workflows.

